In [1]:
import sys

sys.path.append("../src")  # for src imports

In [2]:
import math
import os

import cv2
import folium
import geopandas as gpd
import numpy as np
from IPython.display import display
from owslib.wms import WebMapService
from shapely.geometry import Point

# from wms.helpers import get_image_size_px

In [3]:
gdf = gpd.read_file(
    r"C:\code\cassda-zertifikatsarbeit\data\de_wildlife_crossing.geojson"
)

In [8]:
gdf[gdf["bridge"] == "yes"].count

<bound method DataFrame.count of                     id                @id abandoned:highway access alt_name  \
0    relation/12705178  relation/12705178              None   None     None   
3    relation/13297880  relation/13297880              None   None     None   
4         way/32784047       way/32784047              None   None     None   
5         way/32847260       way/32847260              None   None     None   
6         way/34301551       way/34301551              None   None     None   
..                 ...                ...               ...    ...      ...   
177      way/285884833      way/285884833              None   None     None   
178      way/298051393      way/298051393              None   None     None   
340     way/1026086717     way/1026086717             track   None     None   
365     way/1062603807     way/1062603807              None   None     None   
396     way/1387016220     way/1387016220              None   None     None   

     area barrier 

In [ ]:
# Create a centroids GeoDataFrame from polygon geometries in `gdf`
# Only keep polygon-like geometries; if already points, they will be passed through as-is
centroids = gdf.copy()


# If the geometry is polygon or multipolygon, replace with centroid; otherwise keep geometry
def to_centroid(geom):
    if geom is None or geom.is_empty:
        return geom
    if geom.geom_type in ("Polygon", "MultiPolygon"):
        return geom.centroid
    return geom


centroids["geometry"] = centroids.geometry.apply(to_centroid)
# Ensure result is a GeoDataFrame with proper CRS
centroids = gpd.GeoDataFrame(centroids, geometry="geometry", crs=gdf.crs)
# Optionally drop features where geometry is still None or not a Point
centroids = centroids[centroids.geometry.notnull()]

In [9]:
bounds = gdf.total_bounds  # (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = bounds

center = [(miny + maxy) / 2, (minx + maxx) / 2]
m = folium.Map(location=center, zoom_start=6)
fg = folium.FeatureGroup(name="Wildlife Crossings")
for idx, row in centroids.iterrows():
    geom = row.geometry
    if geom is None or geom.is_empty:
        continue
    # handle MultiPoint by iterating
    if geom.geom_type == "MultiPoint":
        for pt in geom.geoms:
            folium.CircleMarker(
                location=[pt.y, pt.x], radius=4, color="red", fill=True
            ).add_to(fg)
    elif geom.geom_type == "Point":
        folium.CircleMarker(
            location=[geom.y, geom.x], radius=4, color="blue", fill=True
        ).add_to(fg)
fg.add_to(m)
folium.LayerControl().add_to(m)
# Display map in notebook
display(m)

In [11]:
wms = WebMapService(
    "https://image.discomap.eea.europa.eu/arcgis/services/GioLand/VHR_2021_LAEA/ImageServer/WMSServer/?request=GetCapabilities&service=WMS",
    version="1.3.0",
)

In [12]:
layer_name = list(wms.contents)[0]

In [13]:
# create a bbox around the points with some padding
padding = 0.1  # degrees
minx, miny, maxx, maxy = (
    minx - padding,
    miny - padding,
    maxx + padding,
    maxy + padding,
)

In [18]:
# remove exact duplicate geometries
centroids["wkt_tmp"] = centroids.geometry.apply(
    lambda g: g.wkt if g is not None else ""
)
centroids = (
    centroids.drop_duplicates(subset="wkt_tmp")
    .drop(columns="wkt_tmp")
    .reset_index(drop=True)
)

In [ ]:
# Output directory for crops
out_dir = os.path.join("..", "data", "de_wms_export")
os.makedirs(out_dir, exist_ok=True)

In [31]:
# for every point create a 200x200m polygon around it and request a WMS crop centered on the point
img_format = "image/png"
half_size_m = 200  # half of 200 m box

for idx, row in centroids.iterrows():
    print(f"Processing crossing {idx}...")
    geom = row.geometry
    if geom is None or geom.is_empty or geom.geom_type != "Point":
        continue
    point = geom
    # Calculate bbox in lat/lon (EPSG:4326) around the point with half_size_m buffer
    # Approximate conversion from meters to degrees (valid for small distances)
    lat = point.y
    lon = point.x
    delta_deg = half_size_m / 111320  # rough approximation: 1 deg latitude ~ 111.32 km
    bbox = (lon - delta_deg, lat - delta_deg, lon + delta_deg, lat + delta_deg)

    # Get image size in pixels for the bbox at the desired resolution
    img_size = get_image_size_px(1, bbox)

    # Request WMS image
    img = wms.getmap(
        layers=[layer_name],
        srs="EPSG:4326",
        bbox=bbox,
        size=img_size,
        format=img_format,
        transparent=True,
    )

    # Save image to file
    img_data = img.read()
    img_filename = os.path.join(out_dir, f"crossing_{idx}.png")
    with open(img_filename, "wb") as f:
        f.write(img_data)

    print(f"Saved WMS crop for crossing {idx} to {img_filename}")

Processing crossing 0...
Saved WMS crop for crossing 0 to ..\data\de_wms_export\crossing_0.png
Processing crossing 1...
Saved WMS crop for crossing 0 to ..\data\de_wms_export\crossing_0.png
Processing crossing 1...
Saved WMS crop for crossing 1 to ..\data\de_wms_export\crossing_1.png
Processing crossing 2...
Saved WMS crop for crossing 1 to ..\data\de_wms_export\crossing_1.png
Processing crossing 2...
Saved WMS crop for crossing 2 to ..\data\de_wms_export\crossing_2.png
Processing crossing 3...
Saved WMS crop for crossing 2 to ..\data\de_wms_export\crossing_2.png
Processing crossing 3...
Saved WMS crop for crossing 3 to ..\data\de_wms_export\crossing_3.png
Processing crossing 4...
Saved WMS crop for crossing 3 to ..\data\de_wms_export\crossing_3.png
Processing crossing 4...
Saved WMS crop for crossing 4 to ..\data\de_wms_export\crossing_4.png
Processing crossing 5...
Saved WMS crop for crossing 4 to ..\data\de_wms_export\crossing_4.png
Processing crossing 5...
Saved WMS crop for crossi